# Lab 11: Grid Localization using Bayes Filter (Real Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

### <span style="color:rgb(0,150,0)">The notebook only provides skeleton code for you to integrate the Localization class with the Real Robot.</span>

<hr>

In [1]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from localization_extras import Localization

# The imports below will only work if you copied the required ble-related python files 
# into the notebooks directory
from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import numpy as np

import time
import math
import csv
import re

import matplotlib as mpl
import matplotlib.pyplot as plt

# Setup Logger
LOG = get_logger('demo_notebook.log')
LOG.propagate = False

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

2023-04-27 17:01:51,151 | INFO     |: Logger demo_notebook.log initialized.


In [2]:
# Start the plotter
START_PLOTTER()

# The RealRobot class
Define the RealRobot class in the code cell below, based on the documentation and your real robot communication protocol. <br>
This class is used by the **Localization** class to communicate with the real robot. <br>
More specifically, the **Localization** class utilizes the **RealRobot's** member function **perform_observation_loop()** to get the 18 sensor readings and store them in its member variable **obs_range_data**, which is then utilized in the update step.

In [3]:
class RealRobot():
    """A class to interact with the real robot
    """
    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(str(pathlib.Path(os.getcwd()).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble

    def get_pose(self):
        """Get robot pose based on odometry
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    async def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior on the real robot, where the robot does  
        a 360 degree turn in place while collecting equidistant (in the angular space) sensor
        readings, with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count"(=18) defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
                        Do not remove this parameter from the function definition, even if you don't use it.
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               The bearing values are not used in the Localization module, so you may return a empty numpy array
        """
        
        global tof
        global pose
        global CURR_LOG_ARRAY
        
        tof = []
        pose = []
        
        print("starting reading")
        ble.send_command(CMD.ENABLE_BUFFER, "POSE")
        ble.send_command(CMD.ENABLE_BUFFER, "TOF")
        ble.send_command(CMD.SET_PID_GAINS,"ROTATION:|-0.01:|-0.001:|0.0")
        ble.send_command(CMD.ENABLE_ROBOT, "")
        await asyncio.sleep(20)
        ble.send_command(CMD.DISABLE_ROBOT, "")
        ble.send_command(CMD.DISABLE_BUFFER, "POSE")
        ble.send_command(CMD.DISABLE_BUFFER, "TOF")
        ble.start_notify(ble.uuid['RX_STRING'], ble_msg_handler)
        CURR_LOG_ARRAY = tof
        print("Retrieving TOF")
        ble.send_command(CMD.RETRIEVE_BUFFER, "TOF")
        await asyncio.sleep(5)
        print("TOF retrieved")
        CURR_LOG_ARRAY = pose
        print("Retrieving POSE")
        ble.send_command(CMD.RETRIEVE_BUFFER, "POSE")
        await asyncio.sleep(10)
        print("POSE retrieved")
        
        bearings = [20, -20]
        
        tof = np.array(tof).astype('float64')
        pose = np.array(pose).astype('float64')

        ret_tof = np.zeros([18, 2])
        temp = np.zeros([18, 3])

        for i in range(0, 18):
            angle = i * 20
            angle_i = np.argmin(np.abs(pose[:, 3] - angle))
            angle_t = pose[angle_i, 0]
            angle = pose[angle_i, 3]
            tof_reading = tof[np.argmin(np.abs(tof[:, 0] - angle_t)), 1:3] / 1000
            temp[i, :] = [tof_reading[1], tof_reading[0], angle]
            
        ret_tof = temp[:, 0:2]
        print(ret_tof)
            
        return ret_tof, bearings


In [4]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

CURR_LOG_ARRAY = None
tof = []
pose = []
log_complete = False

def ble_msg_handler(sender, msg):
    msg = msg.decode("utf-8")
    end_regex = r'<END BUFFER [a-z, A-Z]*>'
    if re.search(end_regex, msg) != None:
        print("Buffer read complete.")
        log_complete = True
    msg_regex = r'[A-Za-z]*: -?(\d*.?(?:\d*)?)' # Match Data of the form "[name]: [data]"
    data = re.findall(msg_regex, msg)
    if data == None:
        return
    if len(data) == 0:
        return
    CURR_LOG_ARRAY.append(data)

2023-04-27 17:02:02,816 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:83:0c:66:2f:3c
2023-04-27 17:02:05,745 | INFO     |: Connected to c0:83:0c:66:2f:3c


/home/michael/Documents/Cornell/FAST-ROBOTS/FAST-ROBOTS-2023/FastRobots-sim-release/notebooks/base_ble.py:87: FutureWarning: This method will be removed future version, pass the callback to the BleakClient constructor instead.
  self.client.set_disconnected_callback(self.disconnect_handler)


In [5]:
# Initialize RealRobot with a Commander object to communicate with the plotter process
# and the ArtemisBLEController object to communicate with the real robot
robot = RealRobot(cmdr, ble)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

2023-04-27 17:02:10,462 | INFO     |:  | Number of observations per grid cell: 18
2023-04-27 17:02:10,463 | INFO     |:  | Precaching Views...


/home/michael/Documents/Cornell/FAST-ROBOTS/FAST-ROBOTS-2023/FastRobots-sim-release/localization.py:155: RuntimeWarning: All-NaN slice encountered
  return np.nanmin(distance_intersections_tt), intersections_tt[np.nanargmin(distance_intersections_tt)]


2023-04-27 17:02:16,231 | INFO     |:  | Precaching Time: 5.768 secs
2023-04-27 17:02:16,232 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-04-27 17:02:16,232 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107


# Run an update step of the Bayes Filter

In [10]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
await loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
# current_odom, current_gt = robot.get_pose()
# cmdr.plot_gt(current_gt[0], current_gt[1])
# cmdr.plot_odom(current_odom[0], current_odom[1])

2023-04-27 17:15:50,407 | INFO     |: Initializing beliefs with a Uniform Distribution
2023-04-27 17:15:50,408 | INFO     |: Uniform Belief with each cell value: 0.00051440329218107
starting reading
Retrieving TOF
Buffer read complete.
TOF retrieved
Retrieving POSE
POSE retrieved
[[2.416 0.845]
 [2.172 1.029]
 [0.872 1.618]
 [1.434 1.411]
 [1.5   1.394]
 [1.328 1.347]
 [1.448 0.831]
 [1.169 1.102]
 [0.772 1.559]
 [1.485 1.732]
 [1.564 2.057]
 [1.77  1.427]
 [1.917 0.862]
 [1.342 0.996]
 [0.81  1.86 ]
 [1.355 2.613]
 [1.8   2.442]
 [2.421 0.885]]
2023-04-27 17:16:26,239 | INFO     |: Update Step
2023-04-27 17:16:26,242 | INFO     |:      | Update Time: 0.002 secs
2023-04-27 17:16:26,243 | INFO     |: Bel index     : (5, 4, 8) with prob = 0.9832120
2023-04-27 17:16:26,244 | INFO     |: Bel_bar prob at index = 0.00051440329218107
2023-04-27 17:16:26,245 | INFO     |: Belief        : (0.000, 0.000, -10.000)
Buffer read complete.


In [7]:
loc = mapper.to_map(0,0,0)
print(mapper.get_views(loc[0], loc[1], loc[2]))

[[0.52792909 0.46425305]
 [1.79049664 0.46425305]
 [1.45962623 0.52792909]
 [1.3716     1.79049664]
 [1.45962623 1.45962623]
 [1.65964618 1.3716    ]
 [1.23183453 1.45962623]
 [1.08325711 1.65964618]
 [2.0117632  1.23183453]
 [2.28769271 1.08325711]
 [1.79049664 2.0117632 ]
 [0.81090346 2.28769271]
 [1.3716     1.79049664]
 [1.45962623 0.81090346]
 [1.79049664 1.3716    ]
 [1.93573998 1.45962623]
 [0.46425305 1.79049664]
 [0.46425305 1.93573998]]


In [ ]:
print(pose)


In [ ]:
STOP_PLOTTER()